In [1]:
from rap_scrpr import *
from rap_db import *
#nltk.download()
from os import listdir
from os.path import isfile, join

estconn = pg2.connect(database='rap_songs', user='keenan', host='localhost', password='keenan')
#create_music_tables(estconn)

/home/keenan/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
#for show
#this is a full add from start to 
def scrape_multi_artists(art_dic):
    ret_list = []
    for art, page in artist_to_scrape.items():
        #latency timing of 1
        scraped_songs = raw_clean(song_scrape(song_links(page)), art)
        #back up scrape after clean
        with open('json_lyrics/'+art+'_raw.json', 'w') as outfile:
            json.dump(scraped_songs, outfile)
        print(art+'_raw.json made!')
        ret_list.append(art+'_raw.json')
    return ret_list
        
def bulk_load(conn, new_eds = []):
    if not new_eds:
        new_eds = [f for f in listdir('json_lyrics/') if isfile(join('json_lyrics/', f))]
    for new in new_eds:
        songs = json.load(open('json_lyrics/'+new))
        art_name = new.replace("_raw.json","")
        add_base(conn, art_name)
        for name, works in songs.items():
            if 'raw_song_' not in name:
                add_songs(conn, art_name, name, works)
        print(new+" added!")

In [ ]:
#boolean tells it if its one of two style pages, in final product scrape an artist
#artist_to_scrape =  {'chance_the_rapper': ('http://ohhla.com/anonymous/chancetr/', True)}
#HAVE TO RESCRAPE, CLEANING PROCESS TOOK OUT VITAL INFO
artist_to_scrape = {'del':'http://www.ohhla.com/YFA_del.html'}
added = scrape_multi_artists(artist_to_scrape)
#now safely stored in json file filled folder
#create_music_tables(estconn, bypass = True)
#now load them into db
bulk_load(estconn)

create_music_tables(estconn, bypass = True)
bulk_load(estconn, ['mf_doom', 'chance_the_rapper'])

In [5]:
#basic_lyrc_pull(estconn, 'MF Doom')

In [4]:
#
estconn.close()